## Hierarchical classification using Local Classification per Parent Node technique
#### similar logic like LCPN but better structured code for training different classifiers

Here we use the same approach as in /solutions/custom_tokens/xlm_roberta_with_classification_start_span_token.ipynb to train the first classifier to predict the first class, and we train extra three classifiers for each of those classes that will provide us fine-grained classification.

Ovo je pokusaj potpuno odvojenog treniranja klasifikatora koji odreduju finije klase kako bi se lakse upravljalo batch sizeom kod treniranja na drugom nivou, sto se cinilo potrebno, ali se ispostavilo da je jos gore, ILI JE NESTO TOTALNO KRIVO.
Testiranje nije implementirano, jer je vec samo treniranje lose.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
sub1 = 'drive/My Drive/Colab Notebooks/semeval_data/subtask1.parquet'
print(sub1)

# from pathlib import Path
# wd = Path.cwd()
# wd = wd.parent.parent
# wd = wd / 'merged_data'
# sub1 = str(wd) + '/subtask1.parquet'
# print(sub1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/My Drive/Colab Notebooks/semeval_data/subtask1.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)

In [3]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label1'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)

In [4]:
def labelNum2(row):
    labels2 = [0 for _ in range(12)]
    if row['label1'] == 2:
        #labels2 = [0 for _ in range(6)]
        if 'Guardian' in row['classes2']:
            labels2[0] = 1
        if 'Martyr' in row['classes2']:
            labels2[1] = 1
        if 'Peacemaker' in row['classes2']:
            labels2[2] = 1
        if 'Rebel' in row['classes2']:
            labels2[3] = 1
        if 'Underdog' in row['classes2']:
            labels2[4] = 1
        if 'Virtuous' in row['classes2']:
            labels2[5] = 1
    elif row['label1'] == 0:
        #labels2 = [0 for _ in range(12)]
        if 'Instigator' in row['classes2']:
           labels2[0] = 1
        if 'Conspirator' in row['classes2']:
            labels2[1] = 1
        if 'Tyrant' in row['classes2']:
            labels2[2] = 1
        if  'Foreign Adversary' in row['classes2']:
            labels2[3] = 1
        if 'Traitor' in row['classes2']:
            labels2[4] = 1
        if 'Spy' in row['classes2']:
            labels2[5] = 1
        if 'Saboteur' in row['classes2']:
            labels2[6] = 1
        if 'Corrupt' in row['classes2']:
            labels2[7] = 1
        if 'Incompetent' in row['classes2']:
            labels2[8] = 1
        if 'Terrorist' in row['classes2']:
            labels2[9] = 1
        if 'Deceiver' in row['classes2']:
            labels2[10] = 1
        if 'Bigot' in row['classes2']:
            labels2[11] = 1
    elif row['label1'] == 1:
        #labels2 = [0 for _ in range(4)]
        if 'Forgotten' in row['classes2']:
            labels2[0] = 1
        if 'Exploited' in row['classes2']:
            labels2[1] = 1
        if 'Victim' in row['classes2']:
            labels2[2] = 1
        if 'Scapegoat' in row['classes2']:
            labels2[3] = 1
    return labels2

df['label2'] = df.apply(labelNum2, axis=1)

In [5]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)

    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]

    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
df['new_start_end'] = df.apply(adjust_start_end,axis=1)
print(df.loc[0])

lang                                                            BG
art_name                                                BG_670.txt
entity                                                       Запад
start                                                          152
end                                                            156
class1                                                  Antagonist
classes2              [Conspirator, Instigator, Foreign Adversary]
text             Опитът на колективния Запад да „обезкърви Руси...
label1                                                           0
input            Опитът на колективния Запад да „обезкърви Руси...
label2                        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
new_start_end                                           (151, 156)
Name: 0, dtype: object


In [6]:
def addTokensToInput(row):
    inp = row['input']
    start,end = row['new_start_end']
    #print(start,end)
    start = int(start)
    end = int(end)
    token_input = inp[:start] + "[SPAN_START] " + inp[start:end] + " [SPAN_END]" + inp[end:]
    return token_input

df['span_input'] = df.apply(addTokensToInput,axis=1)

In [7]:
def upStartEnd(row):
    start,end = row['new_start_end']
    start += len("[SPAN_START] ")
    end += len("[SPAN_START] ")
    return start,end

df['new_start_end'] = df.apply(upStartEnd,axis = 1)

In [8]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['span_input'], padding=True, truncation=True,max_length=8192,return_offsets_mapping=True)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
extraTokens = {
    "additional_special_tokens": ["[SPAN_START]", "[SPAN_END]"]
}
num_added_toks = tokenizer.add_special_tokens(extraTokens)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(250004, 768, padding_idx=1)

In [10]:
data = df.loc[ : , ['span_input', 'label1', 'label2', 'new_start_end', 'entity']]

In [11]:
data

,span_input,label1,label2,new_start_end,entity
0,Опитът на колективния Запад да „обезкърви Руси...,0,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","(164, 169)",Запад
1,Опитът на колективния Запад да „обезкърви Руси...,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(541, 544)",САЩ
2,Опитът на колективния Запад да „обезкърви Руси...,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(546, 550)",НАТО
3,Опитът на колективния Запад да „обезкърви Руси...,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","(589, 596)",Украйна
4,Опитът на колективния Запад да „обезкърви Руси...,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(644, 661)",украински войници
...,...,...,...,...,...
2897,Медведев: Даже в случае признания поражения Ки...,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(569, 576)",Россией
2898,Медведев: Даже в случае признания поражения Ки...,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(1015, 1021)",Москва
2899,Медведев: Даже в случае признания поражения Ки...,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","(1308, 1312)",НАТО
2900,Медведев: Даже в случае признания поражения [S...,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","(57, 63)",Киевом


In [12]:
data['tokenized']=data.apply(preprocess_function,axis=1)

In [13]:
def indexes(row):
    off_mask = row['tokenized']['offset_mapping']
    start,end = row['new_start_end'][0],row['new_start_end'][1]
    inds = list()
    for p in range(len(off_mask)):
        if off_mask[p][0] >= start and off_mask[p][1] <= end:
            if p != len(off_mask)-1:
                inds.append(p)
    #if len(inds) > 1:
        #print("GREATER THAN 1")
    if len(inds) == 0:
        print(start,end)
    return inds
data['indexes'] = data.apply(indexes,axis=1)

In [14]:
data['list'] = data['tokenized'].apply(lambda x: x['input_ids'])
data['attention'] = data['tokenized'].apply(lambda x: x['attention_mask'])
ids = data['list']
att = data['attention']
indexes = data['indexes']
tids = list()
tatt = list()
print(len(ids),len(att),len(indexes))
for i in range(len(ids)):
    tids.append(torch.tensor(ids[i]))
    tatt.append(torch.tensor(att[i]))

2902 2902 2902


In [15]:
sliced_ids = list()
sliced_ntids = list()
sliced_att = list()
key_inds = list()
key_ids = list()

def slices(index,size,context_size):
    if (size<context_size):
        return 0,size
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    #print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1


for i in range(len(tids)):
    slower,supper = slices(indexes[i][0],len(tids[i]),510)
    #key_tid = tids[i][indexes[i][0]]
    pid = ids[i][slower:supper]
    key_inds.append([])
    for j in indexes[i]:
        key_id = ids[i][j]
        if key_id not in pid:
           print(len(ids[i]),key_id,slower,supper,indexes[i])
        key_inds[i].append(pid.index(key_id))
    apid = tids[i][slower:supper]
    apatt = tatt[i][slower:supper]
    if 0 not in pid:
        apid = torch.cat((torch.tensor([0]),apid),dim=0)
        apatt = torch.cat((torch.tensor([1]),apatt),dim=0)
    if 2 not in pid:
        apid = torch.cat((apid,torch.tensor([2])),dim=0)
        apatt = torch.cat((apatt,torch.tensor([1])),dim=0)
    sliced_ids.append(apid)
    sliced_att.append(apatt)

Min = 10000
Max = 0
ind2 = 0
for i in range(len(indexes)):
    if len(sliced_ids[i]) < Min:
        Min = len(sliced_ids[i])
        ind2 = i

    if len(sliced_ids[i]) > Max:
        Max = len(sliced_ids[i])

In [16]:
input_ids = list()
att_mask = list()
for ten,att in zip(sliced_ids,sliced_att):
    if len(ten) < 512:
        padding_length = 512 - len(ten)
        padding_tensor = torch.full((padding_length,), tokenizer.pad_token_id, dtype=ten.dtype)
        padding_tensor2 = torch.full((padding_length,), 0, dtype=att.dtype)
        ten = torch.cat((ten,padding_tensor),dim=0)
        att = torch.cat((att,padding_tensor2),dim=0)
    input_ids.append(ten)
    att_mask.append(att)
inputIds = torch.stack(input_ids)
attMask = torch.stack(att_mask)

inputIds_np = inputIds.numpy()
attMask_np = attMask.numpy()
y1 = data['label1'].values
y2 = data['label2'].values

In [17]:
from sklearn.model_selection import train_test_split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y1_train, y1_test, y2_train, y2_test = train_test_split(
    inputIds_np, attMask_np, y1, y2, test_size=0.2, random_state=42, shuffle=True
)

In [18]:
import numpy as np
y2_train = np.array(y2_train.tolist(), dtype=np.int8)
y2_test = np.array(y2_test.tolist(), dtype=np.int8)

In [19]:
X_train_ids = torch.tensor(X_train_ids, dtype=torch.long).to(device)
X_test_ids = torch.tensor(X_test_ids, dtype=torch.long).to(device)
X_train_mask = torch.tensor(X_train_mask, dtype=torch.long).to(device)
X_test_mask = torch.tensor(X_test_mask, dtype=torch.long).to(device)
y1_train = torch.tensor(y1_train, dtype=torch.long).to(device)
y1_test = torch.tensor(y1_test, dtype=torch.long).to(device)
y2_train = torch.tensor(y2_train, dtype=torch.long).to(device)
y2_test = torch.tensor(y2_test, dtype=torch.long).to(device)

In [20]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train_ids, X_train_mask, y1_train, y2_train)
test_dataset = TensorDataset(X_test_ids, X_test_mask, y1_test, y2_test )

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [21]:
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn as nn

#first layer classifier
classifier = nn.Linear(model.config.hidden_size, 3).to(device)
optimizer = AdamW(model.parameters(), lr=8e-6)
criterion = nn.CrossEntropyLoss()

In [22]:
import torch
import torch.nn as nn

#second layer classifier
# class SecondLayerClassifier(nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super(SecondLayerClassifier, self).__init__()
#         self.fc = nn.Linear(input_dim, num_classes)
#         self.num_classes = num_classes

#     def forward(self, x):
#         return self.fc(x)

class SecondLayerClassifier(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(SecondLayerClassifier, self).__init__()
        self.transformer_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=8,
            dim_feedforward=2048,
            dropout=0.1,
            activation="gelu"
        )
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_layer, num_layers=1) # could add more layers
        self.classifier = nn.Linear(embedding_dim, num_classes)
        self.num_classes = num_classes
        #self.sigmoid = nn.Sigmoid()

    def forward(self, bert_embeddings):
        # bert_embeddings shape: [batch_size, seq_length, embedding_dim]
        transformer_output = self.transformer_encoder(bert_embeddings)
        pooled_output = transformer_output.mean(dim=1)  # [batch_size, embedding_dim]
        logits = self.classifier(pooled_output)
        return logits #don't return sigmoid(logits) because BCEWithLogitsLoss will be used


In [23]:

second_layer_classifiers = {
    int(0) : SecondLayerClassifier(embedding_dim=model.config.hidden_size, num_classes=12).to(device) , #Antagonist
    int(1) : SecondLayerClassifier(embedding_dim=model.config.hidden_size, num_classes=4).to(device) , #Innocent
    int(2) : SecondLayerClassifier(embedding_dim=model.config.hidden_size,  num_classes=6).to(device) , #Protagonist
}

second_layer_optimizers = {
    name: AdamW(sec_layer_classifier.parameters(), lr=1e-5)
    for name, sec_layer_classifier in second_layer_classifiers.items()
}

second_layer_criterion = {
    name: nn.BCEWithLogitsLoss()
    for name, sec_layer_classifier in second_layer_classifiers.items()
}

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [24]:
import numpy as np
# for the confusion matrix in the end for the first class
all_preds = np.array([], dtype=np.int8)
all_labels = np.array([], dtype=np.int8)

In [25]:
from collections import defaultdict
second_layer_data = defaultdict(list)

num_epochs = 6
#first layer classification on [START_SPAN] token
for epoch in range(num_epochs):

    model.train()
    classifier.train()

    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")

    for batch in train_progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels_1 = batch[2].to(device)
        labels_2 = batch[3].to(device)

        batch_size = input_ids.size(0)

        #taking the output from BERT model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_1, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]

        span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
        start_mask = (input_ids == span_start_token_id)
        entity_representations = []
        start_indices = start_mask.nonzero(as_tuple=True)[1]
        valid_spans = (start_indices != -1)
        valid_start_indices = start_indices[valid_spans]

        for i in range(batch_size):
            entity_tokens = hidden_states[i, valid_start_indices[i]] #for this version only start span tokens
            entity_representations.append(entity_tokens)

        entity_representations = torch.stack(entity_representations, dim=0)

        # not part of training - just grouping data for second-layer classifiers based on labels_1 and after going through bert
        if (epoch == num_epochs - 1):
            for i in range(batch_size):
                label = labels_1[i].item()
                second_layer_data[label].append({
                    'embedded_sequence': hidden_states[i].detach().cpu(),
                    'label_2': labels_2[i].cpu(),
                    # 'input_ids': input_ids[i].cpu(),
                    # 'attention_mask': attention_mask[i].cpu(),
                })

        logits = classifier(entity_representations)
        loss = criterion(logits, labels_1)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(logits, dim=-1)
        correct_predictions += (preds == labels_1).sum().item()
        total_predictions += labels_1.size(0)

        train_progress_bar.set_postfix({'loss': loss.item()})

    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")

    #if (epoch = epoch-1): spremi u dataloader za testiranje - ILI NE MORAS JER JE shuffle = False hehe

    model.eval()
    classifier.eval()

    test_loss = 0
    correct_test_predictions = 0
    total_test_predictions = 0

    test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")

    with torch.no_grad():
        for batch in test_progress_bar:

            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels_1 = batch[2].to(device)
            labels_2 = batch[3].to(device)

            batch_size = input_ids.size(0)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_1, output_hidden_states=True)

            hidden_states = outputs.hidden_states[-1]

            span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
            start_mask = (input_ids == span_start_token_id)
            entity_representations = []
            start_indices = start_mask.nonzero(as_tuple=True)[1]
            valid_spans = (start_indices != -1)
            valid_start_indices = start_indices[valid_spans]

            # extract entity tokens for every sample in batch
            for i in range(batch_size):
                entity_tokens = hidden_states[i, valid_start_indices[i]]
                entity_representations.append(entity_tokens)

            entity_representations = torch.stack(entity_representations, dim=0)

            logits = classifier(entity_representations)
            loss = criterion(logits, labels_1)
            test_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)

            correct_test_predictions += (preds == labels_1).sum().item()
            total_test_predictions += labels_1.size(0)

            #for confusion matrix in determing first class
            all_preds = np.concatenate((all_preds, preds.cpu().numpy()))
            all_labels = np.concatenate((all_labels, labels_1.cpu().numpy()))

            test_progress_bar.set_postfix({'loss': loss.item()})

    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_test_predictions / total_test_predictions

    print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

Training Epoch 1/6: 100%|██████████| 146/146 [03:52<00:00,  1.59s/it, loss=0.101]


Epoch 1/6
Training loss: 0.8543, Training accuracy: 0.6062


Test Epoch 1/6: 100%|██████████| 37/37 [00:17<00:00,  2.10it/s, loss=0.375]


Test loss: 0.5920, Test accuracy: 0.7849


Training Epoch 2/6: 100%|██████████| 146/146 [03:51<00:00,  1.58s/it, loss=0.869]


...

Test loss: 0.5003, Test accuracy: 0.8537


Training Epoch 6/6: 100%|██████████| 146/146 [03:53<00:00,  1.60s/it, loss=0.0193]


Epoch 6/6
Training loss: 0.0972, Training accuracy: 0.9668


Test Epoch 6/6: 100%|██████████| 37/37 [00:17<00:00,  2.10it/s, loss=0.332]

Test loss: 0.5489, Test accuracy: 0.8571


In [26]:
print(len(second_layer_data[0]))
print(len(second_layer_data[1]))
print(len(second_layer_data[2]))



1145
508
668


In [27]:
second_layer_data[2]

[{'embedded_sequence': tensor([[-0.0095,  0.0101, -0.1641,  ..., -0.0786, -0.2833,  0.3536],
          [ 0.8764, -0.4776, -0.3648,  ...,  0.3911, -0.4680,  0.2646],
          [ 1.0914, -0.4429, -0.5957,  ...,  0.8323, -1.1209,  1.0841],
          ...,
          [-0.0867,  0.3878, -0.5374,  ...,  0.7336,  0.0782,  0.3708],
          [-0.0452,  0.1563, -0.1143,  ...,  0.2652, -0.1074,  0.2685],
          [ 0.7581,  0.0733, -0.2775,  ...,  0.3550, -0.2887,  0.4750]]),
  'label_2': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'embedded_sequence': tensor([[ 0.2317,  0.1820,  0.1321,  ..., -0.3270, -0.0375,  0.0253],
          [-0.0083,  0.0203, -0.2792,  ...,  0.9364,  0.1485,  0.4471],
          [-0.3928, -0.3549, -0.4412,  ..., -0.1857,  0.4683,  0.2274],
          ...,
          [-0.3441, -0.0941, -0.2546,  ..., -0.5316,  0.2847,  0.9230],
          [ 0.4030,  0.4658,  0.2679,  ..., -0.2618, -0.5115,  0.2858],
          [ 0.0746,  0.0657,  0.0191,  ..., -0.1229, -0.0518,  0.0620]]),


In [28]:
# creating second layer dataloaders
second_layer_dataloaders = {}
for label, data in second_layer_data.items():
    embedded_sequence = torch.stack([item['embedded_sequence'] for item in data])
    labels_2 = torch.stack([item['label_2'] for item in data])
    # input_ids = torch.stack([item['input_ids'] for item in data])
    # attention_masks = torch.stack([item['attention_mask'] for item in data])

    dataset = TensorDataset(embedded_sequence, labels_2)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    second_layer_dataloaders[label] = dataloader


In [29]:
#training second layer classificators
debug = 0
num_epochs = 5
first_layer_labels = [0, 1, 2]
for first_layer_label in first_layer_labels:

    for epoch in range(num_epochs):
        print(f"Training classifier {first_layer_label}")
        second_layer_classifiers[first_layer_label].train()

        total_loss = 0
        correct_predictions = 0
        total_predictions = 0

        train_progress_bar = tqdm(second_layer_dataloaders[first_layer_label], desc=f"Training Epoch {epoch + 1}/{num_epochs}")

        for batch in train_progress_bar:
            second_layer_optimizers[first_layer_label].zero_grad()
            embedded_sequence = batch[0].to(device)
            labels_2 = batch[1].to(device)


            num_classes = second_layer_classifiers[first_layer_label].num_classes
            labels_2 = labels_2[:, :num_classes]

            logits = second_layer_classifiers[first_layer_label](embedded_sequence)
            loss = second_layer_criterion[first_layer_label](logits, labels_2.float())

            loss.backward()
            second_layer_optimizers[first_layer_label].step()

            total_loss += loss.item()

            preds = (torch.sigmoid(logits) > 0.5).int()
            correct_predictions += ((preds == labels_2.int()).all(dim=1)).sum().item() #strict accuracy
            total_predictions += labels_2.size(0)

            if (debug == 0 and epoch == 3):
              print(f"embedded_sequence: {embedded_sequence.shape}, logits: {logits.shape}, labels_2: {labels_2.shape}")
              print(f"Sample logits: {logits[0]}, Sample labels_2: {labels_2[0]}")
              #TO DO ispisi preds, pa gledaj kolike su vjerojatnosti, pa po tome priblizi prag za rezultat sigmoid fje... nista ti ne znaci logits

            train_progress_bar.set_postfix({'loss': loss.item()})

        if(epoch == 3):
          debug = 1

        avg_train_loss = total_loss / len(train_dataloader)
        train_accuracy = correct_predictions / total_predictions

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Second Layer Classifier {first_layer_label}\nTraining loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")


Training classifier 0


Training Epoch 1/5: 100%|██████████| 72/72 [00:07<00:00,  9.57it/s, loss=0.303]


Epoch 1/5
Second Layer Classifier 0
Training loss: 0.2114, Training accuracy: 0.0052
Training classifier 0


Training Epoch 2/5: 100%|██████████| 72/72 [00:06<00:00, 10.95it/s, loss=0.275]


Epoch 2/5
Second Layer Classifier 0
Training loss: 0.1417, Training accuracy: 0.0000
Training classifier 0


...

Epoch 3/5
Second Layer Classifier 2
Training loss: 0.1137, Training accuracy: 0.0344
Training classifier 2


Training Epoch 4/5: 100%|██████████| 42/42 [00:03<00:00, 10.80it/s, loss=0.355]


Epoch 4/5
Second Layer Classifier 2
Training loss: 0.1124, Training accuracy: 0.0344
Training classifier 2


Training Epoch 5/5: 100%|██████████| 42/42 [00:03<00:00, 10.59it/s, loss=0.359]

Epoch 5/5
Second Layer Classifier 2
Training loss: 0.1119, Training accuracy: 0.0269


In [30]:
# TODO for hierarchical classification
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2])

# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Antagonist', 'Innocent', 'Protagonist'], yticklabels=['Antagonist', 'Innocent', 'Protagonist'])
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix')
# plt.show()